In [1]:
import snscrape.modules.twitter as sntwitter
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd
import multiprocessing
from pathlib import Path

In [2]:
def extract_tweet(since, until, thread_num):
    t_query = f'bist100 since:{since.strftime("%Y-%m-%d")} until:{until.strftime("%Y-%m-%d")}'
    tweets = []
    for tweet in sntwitter.TwitterSearchScraper(t_query).get_items():
        tweets.append([tweet.date, tweet.username, tweet.content])
    t_df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet']).set_index('Date')
    filepath = Path(f'./{since.year}/tweet_part{thread_num}.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)
    t_df.to_csv(filepath)

In [4]:
init_date = date(2021, 1, 1)
end_date = date(2021, 12, 1)
pnum = 0
processes = []
while init_date < end_date:
    s_date = init_date + relativedelta(months=2)
    processes.append(multiprocessing.Process(target=extract_tweet, args=(init_date, s_date, pnum,)))
    processes[pnum].start()
    print(f'Process {init_date.strftime("%Y-%m-%d")} started')
    init_date = s_date
    pnum += 1

Process 2021-01-01 started
Process 2021-03-01 started
Process 2021-05-01 started
Process 2021-07-01 started
Process 2021-09-01 started
Process 2021-11-01 started


In [7]:
for process in processes:
    process.join()